In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
from scipy.stats import spearmanr

from predictability.models import PartialLeastSquares
from predictability.utils import sel_kfold
from predictability.constants import BINARY_RESIDUE_FEATURES, PROJECT_ROOT, DATA_ROOT

In [2]:
results_dir = Path(PROJECT_ROOT / "results/amylase/singles/pls")
results_dir.mkdir(exist_ok=True, parents=True)
seed = 42

In [3]:
property = "activity_dp3"
data = pd.read_csv(DATA_ROOT / "amylase/singles.csv").dropna(subset=BINARY_RESIDUE_FEATURES + [property])

In [4]:
experiment_results = {
    "fold": [],
    "feature": [],
    "belongs": [],
    "spearman_val": [],
    "model": [],
}
prediction_dfs = []
for feature in BINARY_RESIDUE_FEATURES:
    for belongs in [0, 1]:
        df = data[data[feature]==belongs].reset_index()
        for i, (train_inds, test_inds) in enumerate(sel_kfold(df, position_col="residue_number", k=10)):
            split_ids = np.empty(len(df), dtype=object)
            split_ids[train_inds] = "train"
            split_ids[test_inds] = "valid"
            df["split"] = split_ids
            model = PartialLeastSquares()
            model.fit(df[df["split"] == "train"], property)
            predictions_val = model.predict(df[df["split"] == "valid"])
            spearman_val, _ = spearmanr(df[df["split"] == "valid"][property].values, predictions_val)
            experiment_results["fold"].append(i)
            experiment_results["feature"].append(feature)
            experiment_results["belongs"].append(belongs)
            experiment_results["spearman_val"].append(spearman_val)
            experiment_results["model"].append("PLS")

/Users/floris/work/phd/projects/mutation-predictability/conda_env/lib/python3.8/site-packages/sklearn/cross_decomposition/_pls.py:305: UserWarning: Y residual is constant at iteration 7
  warnings.warn(f"Y residual is constant at iteration {k}")


In [5]:
results_df = pd.DataFrame(experiment_results)
results_df.to_csv(results_dir / "scores.csv")